In [102]:
import pandas as pd
import numpy as np
import math
import requests
from yelpapi import YelpAPI
from pandas.io.json import json_normalize
#import sys
#from importlib import reload
#reload(sys)
#sys.setdefaultencoding('utf-8')

Info about the Yelp API
Parameters:
* limit = Number of business results to return. By default, it will return 20. Maximum is 50.
* offset = Offset the list of returned business results by this amount. For example if there are 300 businesses, in the first requests there's a max of 50 restaurants sent back to us, in the second we'll do offset = 50 which will get us the businesses from 50 to 100. These still are considered as different requests


* Client ID

B8rRuDSzKbPnW3jBOwXdZg

* API Key

irBVcEgoI6PeOStlUs77hlL-9r0MA5oa14yFIdtA_mtpe3ChIaN8rAyLcCGgQAFQdfI4cK2ZY84cx3ZFIMh2bDBgIotdi_9yN3Xlp23eE-9HF-Ou5vTrErRnngbIXXYx

To get these, must visit https://www.yelp.com/developers/v3/manage_app and create an account

### Make requests

In [100]:
API_KEY = 'irBVcEgoI6PeOStlUs77hlL-9r0MA5oa14yFIdtA_mtpe3ChIaN8rAyLcCGgQAFQdfI4cK2ZY84cx3ZFIMh2bDBgIotdi_9yN3Xlp23eE-9HF-Ou5vTrErRnngbIXXYx'
#Endpoints : https://www.yelp.com/developers/documentation/v3

MAX_RETURNS = 50
ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
HEADERS = {'Authorization': 'bearer %s' % API_KEY}

#Define the inital parameters
INIT_PARAM = {'offset': 0,
             'limit' : 1,
              'term' : 'food',
             'location':'Chicago'}

In [120]:
# find out how many establishment we can collect 
init_response = requests.get(url=ENDPOINT, params=INIT_PARAM, headers = HEADERS)
nb_estab = response.json()['total']
print('The number of establishments satifying the request is : {0} establishments'.format(nb_estab))

# The YelpAPI only returns 50 establishments at a time, and returns in total up to 1000 items
# the number of requests we'll have to make is ceil(the number of establishments divided by 50)
nb_requests = math.ceil(nb_estab / MAX_RETURNS) if (nb_estab < 1000) else 20
print('We will have to make {0} requests'.format(nb_requests))

The number of establishments satifying the request is : 18100 establishments
We will have to make 20 requests


In [121]:
#create dataframe to contain businesses
businesses = pd.DataFrame()

#Make requests and fill up the dataframe
for req in range(nb_requests):
    # the offset is the starting points in Yelp database
    PARAM = {'offset': req*50,
             'limit' : 50,
              'term' : 'food',
             'location':'Chicago'}
    response = requests.get(url=ENDPOINT, params=PARAM, headers = HEADERS)
    businesses = businesses.append(json_normalize(response.json()['businesses']), ignore_index=True)

C:\Users\lilia\.conda\envs\ada\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


### Clean data

In [161]:
#drop useless columns
businesses = businesses.drop(columns=['image_url', 'transactions', 'url','location.address1', 'location.address2',\
                         'location.address3', 'phone','location.display_address', 'display_phone',\
                         'distance'])

In [173]:
#Only keep establishments in Illinois
businesses = businesses[businesses['location.state'] == 'IL']

In [178]:
print(businesses['location.city'].unique())

#after checking out these locations individually, we find that they are close to Chicago, so we can keep them

['Chicago' 'Burbank' 'CHICAGO' 'Oak Lawn' 'Cicero' 'Elmhurst'
 'Elmwood Park' 'Niles' 'Oak Park' 'Franklin Park' 'River Grove'
 'Glendale Heights' 'Forest Park']


In [179]:
businesses.to_pickle("yelp_businesses_pickle")